# Notes from David Beazley's Python3 Metaprogramming tutorial (2013)

- "ported" to Python 2.7, unless noted otherwise

# A Debugging Decorator

In [1]:
from functools import wraps

In [48]:
def debug(func):
    msg = func.__name__
    # wraps is used to keep the metadata of the original function
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

In [49]:
@debug
def add(x,y):
    return x+y

In [50]:
add(2,3)

add


5

In [51]:
def add(x,y):
    return x+y

In [52]:
debug(add)

<function __main__.add>

In [53]:
debug(add)(2,3)

add


5

# Decorators with arguments

## Calling convention

```python
@decorator(args)
def func():
    pass
```

## Evaluation

```python
func = decorator(args)(func)
```

In [54]:
def debug_with_args(prefix=''):
    def decorate(func):
        msg = prefix + func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [55]:
@debug_with_args(prefix='***')
def mul(x,y):
    return x*y

In [56]:
mul(2,3)

***mul


6

In [58]:
def mul(x,y):
    return x*y

In [60]:
debug_with_args(prefix='***')

<function __main__.decorate>

In [61]:
debug_with_args(prefix='***')(mul)

<function __main__.mul>

In [62]:
debug_with_args(prefix='***')(mul)(2,3)

***mul


6

# Decorators with arguments: a reformulation

- TODO: show what happens without the partial application to itself!

In [40]:
from functools import wraps, partial

def debug_with_args2(func=None, prefix=''):
    if func is None: # no function was passed
        return partial(debug_with_args2, prefix=prefix)
    
    msg = prefix + func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

In [63]:
@debug_with_args2(prefix='***')
def div(x,y):
    return x / y

In [64]:
div(4,2)

***div


2

In [70]:
def div(x,y):
    return x / y

In [71]:
debug_with_args2(prefix='***')

In [72]:
debug_with_args2(prefix='***')(div)

<function __main__.div>

In [73]:
debug_with_args2(prefix='***')(div)(4,2)

***div


2

In [42]:
f = debug_with_args2(prefix='***')

In [44]:
def div(x,y):
    return x / y

In [47]:
debug_with_args2(prefix='***')(div)

<function __main__.div>

# Class decorators

- decorate all methods of a class at once
- NOTE: only instance methods will be wrapped, i.e. this won't work with static- or class methods

In [76]:
def debugmethods(cls):
    for name, val in vars(cls).items():
        if callable(val):
            setattr(cls, name, debug(val))
    return cls

In [85]:
@debugmethods
class Spam(object):
    def foo(self):
        pass
    def bar(self):
        pass

In [86]:
s = Spam()

In [87]:
s.foo()

foo


In [88]:
s.bar()

bar


# Class decoration: debug access to attributes

In [96]:
def debugattr(cls):
    orig_getattribute = cls.__getattribute__
    
    def __getattribute__(self, name):
        print('Get:', name)
        return orig_getattribute(self, name)
    cls.__getattribute__ = __getattribute__
    
    return cls

In [97]:
@debugattr
class Ham(object):
    def foo(self):
        pass
    def bar(self):
        pass

In [105]:
h = Ham()

In [107]:
h.foo()

('Get:', 'foo')


In [108]:
h.bar

('Get:', 'bar')


<bound method Ham.bar of <__main__.Ham object at 0x7f4854a86110>>

# Debug all the classes?

- TODO: this looks Python3-specific

## Solution: A Metaclass

In [112]:
class debugmeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super(cls).__new__(cls, clsname, bases, clsdict)
        clsobj = debugmethods(clsobj)
        return clsobj

In [117]:
# class Base(metaclass=debugmeta): # won't work in Python 2.7
#     pass

# class Bam(Base):
#     pass

# cf. minute 27